# Chapter 9: Binary Trees

## Notes:

* Some problems have simple brute force solutions that require `O(n)` space, but, subtler solutions exist that use existing tree nodes to reduce space complexity to `O(1)`.

* Recursive algorithms are well-suited to tree problems. Remember to consider the size of the function call stack which computing space complexity.

* The height of the tree varies from `log n` for balanced trees to `n` for skewed trees.

* If each node has a parent field, use it to reduce time and space complexities.

In [7]:
from collections import namedtuple, deque

class BinaryTreeNode:
    """
    Represents a node in a binary tree
    """
    def __init__(self, data=None, left=None, right=None):
        this.data = data
        this.left = left
        this.right = right


def traversal(tree_node):
    """
    Demonstrates preorder, inorder and postorder traversal
    of a tree rooted at `tree_node`
    """
    print("Pre-order: ", tree_node.data)
    traversal(tree_node.left)
    
    print("In-order: ", tree_node.data)
    traversal(tree_node.right)
    
    print("Post-order: ", tree_node.data)

## Traversals

Pre-order

In [6]:
def preorder_iter(root):
    """
    Returns a list of given tree's nodes as seen during a pre-order traversal.
    """
    stack, res = [root], []
    while stack:
        node = stack.pop()
        if node:
            res.append(node.data)
            stack += [node.right, node.left]
    return res


def preorder_recur(root):
    """
    Returns a list of given tree's nodes as seen during a pre-order traversal.
    """
    res = []
    def helper(node):
        if not node: return
        res.append(node.data)
        helper(node.left)
        helper(node.right)
    helper(root)
    return res

In-order

In [7]:
def in_order_recur(root):
    """
    Returns a list of given tree's nodes as seen during an in-order traversal.
    """
    res = []
    def helper(node):
        if not node: return
        helper(node.left)
        res.append(node.data)
        helper(node.right)
    helper(root)
    return res

def in_order_inorder(root):
    """
    Returns a list of given tree's nodes as seen during an in-order traversal.
    """
    stack, res = [], []
    node = root
    while node or stack:
        while node:
            stack.append(node)
            node = node.left
        node = stack.pop()
        res.append(node.data)
        node = node.right
    return res

Post-order

In [8]:
def post_order_recur(root):
    """
    Returns a list of given tree's nodes as seen during a post-order traversal.
    """
    res = []
    def helper(node):
        if not node: return
        helper(node.left)
        helper(node.right)
        res.append(node.data)
    helper(root)
    return res

def post_order_iter(root):
    """
    Returns a list of given tree's nodes as seen during a post-order traversal.
    """
    if not root: return None
    stack, res = [root], []
    prev = None
    while stack:
        curr = stack[-1]
        if not prev or curr is prev.left or curr is prev.right:
            if curr.left: stack.append(curr.left)
            elif curr.right: stack.append(curr.right)
        elif prev is curr.left:
            if curr.right: stack.append(curr.right)
        else:
            res.append(stack.pop().data)
        prev = curr
    return res

Level-order

In [9]:
def level_order(root):
    """
    Returns a list of given tree's nodes as seen during a level-order traversal.
    """
    if not root: return []
    
    res = []
    curr_level = [root]
    
    while curr_level:
        res.append([node.data for node in curr_level])
        curr_level = [child for node in curr_level for child in (node.left, node.right) if child]
#         curr_level = []
#         for node in curr_level:
#             if node.left: curr_level.append(node.left)
#             if node.right: curr_level.append(node.eight)
    
    return res

## 9.1 Test if a binary tree is height balanced

In [1]:
def is_balanced_tree(tree):
    """
    Returns True iff the given binary tree is height-balanced
    """
    BalancedWithHeight = namedtupled('BalancedWithHeight', ('balanced', 'height'))
    
    def check_balanced(tree):
        if not tree:
            return BalancedWithHeight(True, -1)
    
        left_result = check_balanced(tree.left)
        if not left_result.balanced:
            return BalancedWithHeight(False, 0)
        
        right_result = check_balanced(tree.right)
        if not right_result.balanced:
            return BalancedWithHeight(False, 0)
        
        is_balanced = abs(left_result.height - right_result.height) <= 1
        height = max(left_result.height, right_result.height) + 1
        return BalancedWithHeight(is_balanced, height)
    
    return check_balanced(tree).balanced

### Variant: Design an algorithm that takes as input a binary tree and positive integer k, and returns the node in the binary tree such that the node is not k-balanced, but all of its descendants are k-balanced

In [3]:
def k_balanced(tree, k):
    BalancedWithHeight = namedtuple('BalancedWithHeight', ('balanced', 'height'))
    res = None
    
    def k_balanced_node(tree):
        if not tree:
            return BalancedWithHeight(True, -1)
        left_result = k_balanced_node(tree.left)
        right_result = k_balanced_node(tree.right)
        
        if not left_result.balanced or not right_result.balanced:
            return BalancedWithHeight(False, None)
        
        is_balanced = abs(left_result.height - right_result.height) <= k
        height = max(left_result.height, right_result.height) + 1
        
        nonlocal res
        if not is_balanced and not res:
            res = tree
            
        return BalancedWithHeight(is_balanced, height)
    
    k_balanced_node(tree)
    return res

## 9.2 Test if a binary tree is symmetric

In [9]:
def is_symmetric(tree):
    """
    Returns True iff the given binary tree is symmetric
    """
    
    def check_symmetric(left_subtree, right_subtree):
        if not left_subtree and not right_subtree:
            return True
        elif left_subtree and right_subtree:
            return (left_subtree.data == right_subtree.data) and \
        check_symmetric(left_subtree.left, right_subtree.right) and \
        check_symmetric(left_subtree.right, right_subtree.left)
        return False

    return not tree or check_symmetric(tree.left, tree.right)

## 9.3 Compute the lowest common ancestor in a binary tree

In [4]:
def lca(root, node0, node1):
    """
    Returns the LCA of the given nodes in the given tree
    """
    Status = namedtuple("Status", ("node_count", "ancestor"))
    
    def lca_helper(tree, node0, node1):
        if not tree:
            return Status(0, None)
        
        left_result = lca_helper(tree.left, node0, node1)
        if left_result.node_count == 2:
            return left_result
        
        right_result = lca_helper(tree.right, node0, node1)
        if right_result.node_count == 2:
            return right_result
        
        node_count = left_result.node_count + right_result.node_count + int(tree is node0) + int(tree is node1)
        ancestor = tree if node_count == 2 else None
        return Status(node_count, ancestor)
    
    return lca_helper(root, node0, node1).ancestor

## 9.4 Compute the LCA when nodes have parent pointers

In [13]:
class Node:
    """
    Represents a binary tree node with a parent pointer
    """
    def __init__(self, data=None, parent=None, left=None, right=None):
        this.data = data
        this.parent = parent
        this.left = left
        this.right = right

def lca(node1, node2):
    """
    Returns the LCA of node1 and node2 if it exists
    """
    
    def get_depth(node):
        """
        Returns the depth of the given node
        """
        d = 0
        while node and node.parent: 
            d += 1
            node = node.parent
        return d
    
    def move_up(node, n):
        """
        Returns the given node moved up by n steps
        """
        while n > 0:
            node = node.parent
            n -= 1
        return node
    
    if not node1 or not node2:
        return None
    
    # Compute depths
    depth1 = get_depth(node1)
    depth2 = get_depth(node2)
    
    # Assign the node to be ascended
    if depth2 > depth1:
        node1, node2 = node2, node1
    
    # Ascends the deeper node
    node1 = move_up(node1, abs(depth1 - depth2))
    
    # Ascend both nodes until we reach the LCA
    while node1 is not node2:
        node1, node2 = node1.parent, node2.parent
    return node1

## 9.11 Implement an in-order traversal with `O(1)` space

In [5]:
class Node:
    """
    Represents a binary tree node with a parent pointer
    """
    def __init__(self, data=None, parent=None, left=None, right=None):
        this.data = data
        this.parent = parent
        this.left = left
        this.right = right
        
        
def inorder_traversal_iter(tree):
    """
    Perform in-order traversal iteratively
    """
    prev, result = None, []
    while tree:
        if prev is tree.parent:
            if tree.left:
                next = tree.left
            else:
                result.append(tree.data)
                next = tree.right or tree.parent
        elif prev is tree.left:
            result.append(tree.data)
            next = tree.right or tree.parent
        else:
            next = tree.parent
        prev, tree = tree, next
    return result

## 9.12 Reconstruct a binary tree from traversal data

In [6]:
def build_binary_tree_from_preorder_inorder(preorder, inorder):
    node_to_inorder_idx = {data: i for i, data in enumerate(inorder)}
    
    def binary_tree_from_preorder_inorder_helper(preorder_start, preorder_end, inorder_start, inorder_end):
        if preorder_end <= preorder_start or inorder_end <= inorder_start:
            return None
        root_inorder_idx = node_to_inorder_idx[preorder[preorder_start]]
        left_subtree_size = root_inorder_idx - inorder_start
        return BinaryTreeNode(preorder[preorder_start],
                            # left subtree
                            binary_tree_from_preorder_inorder_helper(
                                preorder_start + 1,
                                preorder_start + 1 + left_subtree_size,
                                inorder_start,
                                root_inorder_idx),
                            # right subtree
                            binary_tree_from_preorder_inorder_helper(
                                preorder_start + 1 + left_subtree_size,
                                preorder_end,
                                root_inorder_idx + 1,
                                inorder_end))
    return binary_tree_from_preorder_inorder_helper(0, len(preorder), 0, len(inorder))

A much shorter solution

In [8]:
def build_binary_tree_from_preorder_inorder_short(preorder, inorder):
    node_to_inorder_idx = {data: i for i, data in enumerate(inorder)}
    preorder = deque(preorder)
    
    def helper(preorder, inorder):
        if not inorder or not preorder:
            return None
        
        inorder_root_idx = node_to_inorder_idx[preorder.popleft()]
        return BinaryTreeNode(preorder[inorder_root_idx],
                                # left subtree
                                helper(preorder, inorder[:inorder_root_idx]),
                                # right subtree
                                helper(preorder, inorder[inorder_root_idx + 1:])) 

### Variant: Solve the same problem with an inorder traversal sequence and a post order traversal sequence

In [9]:
def build_binary_tree_from_postorder_inorder_short(postorder, inorder):
    node_to_inorder_idx = {data: i for i, data in enumerate(inorder)}
    
    def helper(postorder, inorder):
        if not postorder or not inorder:
            return None
        inorder_root_idx = node_to_inorder_idx[postorder.pop()]
        return BinaryTreeNode(postorder[inorder_root_idx],
                              # left subtree
                              helper(postorder, inorder[inorder_root_idx+1:]),
                              # right subtree
                              helper(postorder, inorder[:inorder_root_idx]))